In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose #this is where we are importing the pre-built pose detection model from mediapipe


In [3]:
from IPython.display import HTML

# Define the HTML content
html_content = '<image src="https://i.imageur.com/3j8BPdc.png" style="height:300px">'

# Display the HTML content
display(HTML(html_content))


In [4]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [5]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

landmarks = ['class']
for val in range(1, 34): #because 33 points
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]


In [45]:
cap = cv2.VideoCapture('overHeadSquat.mp4')

feedback = 'hi'
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        width  = cap.get(3)  # float `width`
        height = cap.get(4)  # float `height`
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y] 
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y] 
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y] 
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y] 

            angle = calculate_angle(wrist, hip, knee)
            angle_formatted = "{:.2f}".format(angle)
            angle2 = calculate_angle(hip, knee, ankle)
            angle2_formatted = "{:.2f}".format(angle2)
            angle_diff = float(angle_formatted) - float(angle2_formatted)
            angle_diff_formatted = "{:.2f}".format(angle_diff)

            if float(angle2_formatted) < 80:
                if angle_diff < -10:
                    feedback = 'You are leaning forward, keep chest up' 
                else:
                    feedback = 'goodform'
            else:
                feedback = ''

            print(feedback)
            # Visualize angle
            cv2.putText(image, str(angle_formatted), 
                           tuple(np.multiply(hip, [400, 800]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            cv2.putText(image, str(angle2_formatted), 
                           tuple(np.multiply(knee, [400, 800]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            # cv2.putText(image, str(angle_diff_formatted), 
            #                tuple(np.multiply(knee, [200, 800]).astype(int)), 
            #                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
            #                     )
            
        except Exception as e:
            print(e)
            
        CUSTOM_POSE_CONNECTIONS = [
                # (mp_pose.PoseLandmark.RIGHT_WRIST, mp_pose.PoseLandmark.RIGHT_ELBOW), #16, 14
                # (mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_SHOULDER), #14
                # (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_HIP), #12, 24
                # (mp_pose.PoseLandmark.RIGHT_ANKLE, mp_pose.PoseLandmark.RIGHT_KNEE) #28, 26
                # (mp_pose.PoseLandmark.LEFT_WRIST, mp_pose.PoseLandmark.LEFT_ELBOW), #15, 13
                # (mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_SHOULDER), #13, 11
                # (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_HIP), #11,23
                # (mp_pose.PoseLandmark.LEFT_ANKLE, mp_pose.PoseLandmark.LEFT_KNEE), #27, 25
                (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_WRIST),
                (mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE)
            ]

        
        # Drawing rectangles and text on the image
        cv2.rectangle(image, (0,0), (400,60), (245,117,16), -1)
        
        # Drawing predicted class
        cv2.putText(image, "FEEDBACK", (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
        cv2.putText(image, str(feedback), (15,30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
       
       # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, CUSTOM_POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

















goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform

























































































goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
goodform
good

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
